In [1]:
# 1. Get 10,000 feeds - Assignment 5

In [ ]:
import webhoseio
webhoseio.config(token="2c45808b-852d-4156-af2f-faf960992ea9")
query_params = {"q": "organization:Apple", #change to other org to get 10,000 feeds
                "sort": "crawled"}
output = webhoseio.query("filterWebContent", query_params)
print(output['posts'][0]['text']) # Print the text of the first post
print(output['posts'][0]['published']) # Print the text of the first post publication date

# Print the site of the first post
print(output['posts'][0]['thread']['site'])

feeds = []
for item in output['posts']:
    feeds.append(item)
    #print(item['title'])
print(len(feeds))

count = 100
while count > 0:    
    output = webhoseio.get_next()
    for item in output['posts']:
        feeds.append(item)
    count -= 1
#     if len(feeds) > 10000:
#         break
# feeds = feeds[:10000]
print(len(feeds)) #try changed org to get 10,000 feeds

import json
with open("anly540_final.json", "w") as myfile: #change file name to other org
    for feed in feeds:
        line = json.dumps(feed)
        myfile.write(line)
        myfile.write("\n")
        
json_data = open("anly540_apple.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))
print(len(feeds_read_from_file))

for feed in feeds_read_from_file[:10]:
    print(feed['title'], '\n')
    print(feed['text'], '\n')
    print(feed['url'], '\n')
#print(feeds_read_from_file[0]['title'])


In [ ]:
# 2. 